In [1]:
import glob
import os

import numpy as np
import piexif
import tifffile as tiff
from PIL import Image
from tqdm.notebook import tqdm

from thermal_parser import Thermal

# Extract thermal data from DJI cameras

DJI thermal cameras produce 3-band RGB images in `.JPG` format. However, the thermal data itself (i.e. temperature values) are actually stored in some weird proprietary binary format embedded within the EXIF data. This can be seen using `exiftool`, which is now installed in the Hub environment:

    $ exiftool ./raw_images/DJI_20230706110817_0001_T.JPG 
    [Truncated output]
    Thermal Data                    : (Binary data 655360 bytes, use -b option to extract)
    Thermal Calibration             : (Binary data 32768 bytes, use -b option to extract)
    [Truncated output]

The repository [here](https://github.com/SanNianYiSi/thermal_parser) provides code for extracting the thermal data, which requires the DJI SDK. However, there are somes issues with the code (syntax errors etc.) and the install script doesn't seem to work as intended. I have therefore created a fork of the repository [here](https://github.com/SeaBee-no/thermal_parser) where I have fixed the errors and simplified the installation procedure (mostly by hard-coding some file paths for our environment on Sigma2 so that the library can find the SDK).

The post [here](https://community.opendronemap.org/t/lens-calibration-of-thermal-camera-dji-3t/19798/3) provides an overview of the basic workflow, which is as follows:

 1. Use `thermal_parser` to extract the thermal data as a numpy array. This is then saved as a single-band TIFF (because JPGs don't support float values). However, this conversion process loses all the EXIF data from the original JPGs.
    
 2. Transfer the EXIF data from the original JPGs to the TIFFs. The recommended solution is to use `exiftool`, but it seems like `piexif` can do the same thing from within Python, which is neater.

Step 1 seems to work OK and step 2 completes successfully, but ODM does not recognise the EXIF data that is transferred to the TIFFs. This causes the orthorectification step to fail, which is strange because it works in the thread linked above. I have tried both `exiftool` and `piexif`. Both seem to transfer the EXIF data successfully and produce the same output. More investigation needed...

In [2]:
# Mission folder to process
mission_fold = r"/home/notebook/temp/fedje_stormarkIR_20230706_test"

In [3]:
# Convert 3-band RGB images to single-band thermal images
raw_img_dir = os.path.join(mission_fold, "raw_images")
proc_img_dir = os.path.join(mission_fold, "images")
if not os.path.exists(proc_img_dir):
    os.makedirs(proc_img_dir)

jpg_list = glob.glob(f"{raw_img_dir}/*_T.JPG")
mins = []
maxs = []
for jpg_path in tqdm(jpg_list):
    fname = os.path.basename(jpg_path)

    # Extract thermal data
    thermal = Thermal(dtype=np.float32)
    data = thermal.parse(filepath_image=jpg_path)

    # Get min and max temps
    mins.append(np.nanmin(data))
    maxs.append(np.nanmax(data))

    # Save as .tif
    tif_path = os.path.join(mission_fold, "images", fname[:-4] + ".tif")
    tiff.imwrite(tif_path, data)

    # # Option 1: Copy EXIF info using piexif
    # jpg_image = Image.open(jpg_path)
    # exif_data = piexif.load(jpg_image.info["exif"])
    # tif_image = Image.open(tif_path)
    # tif_image.save(tif_path, exif=piexif.dump(exif_data))

print("Min. temp:", min(mins))
print("Max. temp:", max(maxs))

  0%|          | 0/457 [00:00<?, ?it/s]

Min. temp: 1.7000122
Max. temp: 38.399994


In [4]:
# Option 2: Copy EXIF data using exiftool
src_paths = f"{mission_fold}/raw_images/%f.JPG"
dst_dir = f"{mission_fold}/images/"
!exiftool -overwrite_original -tagsfromfile {src_paths} -all:all {dst_dir}

    1 directories scanned
  457 image files updated
